In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# URLs for datasets
datasets = {
    "SO2TONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/SO2TONS_dataset.csv",
    "NOXTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/NOXTONS_dataset.csv",
    "COTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/COTONS_dataset.csv"
}

# Define the peak season months (May through August)
peak_season_months = [5, 6, 7, 8]

# Define lakes (sources)
sources = ["LAKE-1", "LAKE-2", "LAKE-3", "LAKE-4"]

# Define the specific day for prediction
specific_date = pd.Timestamp("2022-07-15")

# Initialize a dictionary to store models, predictions, and inputs for verification
models = {}
predictions = {}

# Define the parameter grid for GridSearchCV
param_grid = {
    "n_neighbors": list(range(1, 21)),
    "weights": ["uniform", "distance"],
    "p": [1, 2]  # 1: Manhattan distance, 2: Euclidean distance
}

# Loop through each dataset (SO2TONS, NOXTONS, COTONS)
for parameter, url in datasets.items():
    # Load the dataset
    data = pd.read_csv(url)

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter for peak season
    data = data[data['date'].dt.month.isin(peak_season_months)]

    # Separate data by source
    for source in sources:
        source_data = data[data['Source'] == source]

        # Check if the source data has enough rows
        if source_data.empty or len(source_data) < 10:
            print(f"Not enough data for {parameter} at {source}. Skipping...")
            continue

        # Define predictors (e.g., weather features)
        predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
        target = 'Emissions_Load'

        # Drop rows with missing values
        source_data = source_data.dropna(subset=predictors + [target])

        # Split the data into features (X) and target (y)
        X = source_data[predictors]
        y = source_data[target]

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        # Set up the KNN regressor and GridSearchCV for hyperparameter tuning
        knn = KNeighborsRegressor()
        grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Retrieve the best estimator
        best_model = grid_search.best_estimator_

        # Evaluate the tuned model
        y_pred = best_model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"Best parameters for {parameter} at {source}: {grid_search.best_params_}")
        print(f"Model for {parameter} at {source}:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R²: {r2:.4f}")

        # Save the best model and scaler
        models[(parameter, source)] = (best_model, scaler)

        # Check if the specific date exists in the source data
        day_data = source_data[source_data['date'] == specific_date]
        if not day_data.empty:
            # Extract feature values for the specific day
            specific_features = scaler.transform(day_data[predictors])
            specific_actual = day_data[target].iloc[0]

            # Predict emissions/load for the specific day using the tuned model
            specific_prediction = best_model.predict(specific_features)[0]

            # Save the prediction and actual value for verification
            predictions[(parameter, source)] = {
                "features": day_data[predictors].iloc[0],
                "actual": specific_actual,
                "predicted": specific_prediction
            }

# Display all predictions at the end
print("\nFinal Predictions:")
for key, value in predictions.items():
    parameter, source = key
    print(f"{parameter} at {source}:")
    print(f"  Features: {value['features'].to_dict()}")
    print(f"  Actual Emissions_Load: {value['actual']:.4f}")
    print(f"  Predicted Emissions_Load: {value['predicted']:.4f}")
    print()


Best parameters for SO2TONS at LAKE-1: {'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Model for SO2TONS at LAKE-1:
  RMSE: 0.0000
  R²: 0.3280
Best parameters for SO2TONS at LAKE-2: {'n_neighbors': 19, 'p': 1, 'weights': 'distance'}
Model for SO2TONS at LAKE-2:
  RMSE: 0.0000
  R²: 0.1089
Best parameters for SO2TONS at LAKE-3: {'n_neighbors': 12, 'p': 1, 'weights': 'uniform'}
Model for SO2TONS at LAKE-3:
  RMSE: 0.0000
  R²: 0.1879
Best parameters for SO2TONS at LAKE-4: {'n_neighbors': 20, 'p': 2, 'weights': 'distance'}
Model for SO2TONS at LAKE-4:
  RMSE: 0.0000
  R²: 0.2822
Best parameters for NOXTONS at LAKE-1: {'n_neighbors': 12, 'p': 1, 'weights': 'distance'}
Model for NOXTONS at LAKE-1:
  RMSE: 0.0002
  R²: 0.2112
Best parameters for NOXTONS at LAKE-2: {'n_neighbors': 20, 'p': 1, 'weights': 'distance'}
Model for NOXTONS at LAKE-2:
  RMSE: 0.0002
  R²: -0.0796
Best parameters for NOXTONS at LAKE-3: {'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
Model for NOXTONS at LAKE-3:
